In [ ]:
#tensorflow

##首先定义计算图，然后交给C++执行
##支持并行计算；支持分布式计算。
#跨平台；
#简单的API:tensorflow.contrib.learn;skflow;tensorflow.contrib.slim
#高级封装：Keras;Pretty Tensor
#支持灵活的结构设计
#结合C++的高执行效率
#支持先进的优化方法:支持automatic differentiating
#支持tensorboard等可视化工具
#连接谷歌云服务
#强大的社区支持。



#定义计算图
import tensorflow as tf

tf.reset_default_graph()

x=tf.Variable(3,name='x')
y=tf.Variable(4,name='y')
f=x*x*y+y+2

#开启会话执行
sess=tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result=sess.run(f)
print(result)
sess.close()

#利用with管理会话
with tf.Session() as sess:
    x.initializer.run()#相当于调用tf.get_default_session().run(x.initializer)
    y.initializer.run()
    result=f.eval()#相当于调用tf.get_defualt_session().run(f)
    
#利用tf.global_variables_initializer()开启全局初始化
init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result=f.eval()
    
#开启InterativeSession，自动将其设为默认会话，不必启用with
sess=tf.InterativeSession()
init.run()
result=f.eval()
print(result)
sess.close()

#执行阶段常以循环的方式进行训练


#管理图
#所有创建的节点都会自动加入默认图
x1=tf.Variable(1)
x1.graph is tf.get_default_graph()

#若要创建多个图，用with关键词
graph=tf.Graph()
with graph.as_default():
    x2=tf.Variable(2)
x2.graph is graph
    
x2.praph is tf.get_default_graph()

#若多次运行同一些命令，须重启graph，即执行tf.reset_default_graph()

#计算节点值时，tensorflow自动确定节点间的依赖关系
w=tf.constant(3)
x=w+2
y=x+5
z=x*3
with tf.Session() as sess:
    print(y.eval)
    print(z.eval)#用eval会自动计算w，x两次
#节点值在计算后就会被丢弃，但变量不会，会保留到会话结束。

#用run可以让tf一次性计算y,z的值
with tf.Session() as sess:
    y_val,z_val=sess.run([y,z])
    print(y_val)
    print(z_val)
#在单过程的tf，多个会话间并不共享其状态，即使是复用相同的图；但在分布式tensorflow中，变量
#状态存储在服务器中而非会话中，因此，多个会话间可以共享变量状态。

#Linear Regression
#算子可以处理多个输入输出，Constant和Varable不需要输入，称为源算子。而非源算子的输入输出是
#多维数组，称为tensor，Python API中的tensor可有NumPy的数组表示。

import numpy as np
from sklearn.datasets import fetch_california_housing
housing=fetch_california_housing()
m,n=housing.data.shape
housing_data_plus_bias=np.c_[np.ones((m,1)),housing.data]
X=tf.constant(housing_data_plus_bias,dtype=tf.float32,name='X')
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name='y')
XT=tf.transpose(X)
theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)
with tf.Session() as sess:
    theta_value=theta.eval()
    theta_run=sess.run(theta)
    print(theta_run)
    print(theta_value)
    
#手动执行梯度下降
n_epchs=1000
learning_rate=0.01
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name='X')
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name='y')
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name='theta')
y_pred=tf.matmul(X,theta,name='predictions')
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name='mse')
gradients=2/m*tf.matmul(tf.transpose(X),error)
training_op=tf.assign(theta,theta-learning_rate*gradients)
init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print('Epoch',epoch,'MSE',mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()

#自动执行符号微分，从而进行梯度下降
gradients=tf.gradients(mse,[theta])[0]
#返回算子对变量的列表（如果有多个变量）
training_op=tf.assign(theta,theta-learning_rate*gradients)
init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print('Epoch',epoch,'MSE',mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()

#添加优化算子
gradients=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

training_op=optimizer.minimize(mse)
init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print('Epoch',epoch,'MSE',mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()
    
    
    
from datetime import datetime
now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir='tf_logs'
logdir='{}/run-{}/'.format(root_logdir,now)
#为每次运行的结果设一个路径，以防多次运行加载到tensorboard时错误

    
#使用自定义优化算子
optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
gradients=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

training_op=optimizer.minimize(mse)
init=tf.global_variables_initializer()

saver=tf.train.Saver({'weight':theta})#将所有参数初始化后保存所有节点
#可以重命名节点

with tf.Session() as sess:
    
    sess.run(init)
    #如果再次加载，则需重新加载已保存的节点
    saver.restore(sess,'/my_final_model.ckpt')
    
    for epoch in range(n_epochs):
        if epoch%100==0:
            save_path=saver.save(sess,'/my_model.ckpt')
            print('Epoch',epoch,'MSE',mse.eval())#可以在任何时间点保存节点
        sess.run(training_op)
        
    best_theta=theta.eval()
    save_path=saver.save(sess,'/my_final_model.ckpt')#以不同的节点命名不同的保存内容
    mse_summary=tf.summary.scalar("MSE",mse)#创建MSE的可视化节点，并写入summary的二进制tb文件
    file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())
    #将存储的节点写入文件，第二个参数是要可视化的部分。
    #FileWriter会创建登录工作区，写入图的定义至事件文件。
    

#向训练算法添加数据
#使用Mini-batch SGD,需要运用placeholder占位符节点，占位符不进行运算，而是返回数据以返回运行时
#使用None表示任何形状
A=tf.placeholder(tf.float32,shape=(None,3))
B=A+5
with tf.Session() as sess:
    B_val_1=B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2=B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})
    print(B_val_1)
    print(B_val_2)
    
    



In [3]:
X=tf.palceholder(tf.float32,shape=(None,n+1),name='X')
y=tf.placeholder(tf.float32,shape=(None,1),name='y')
batch_size=100
n_batches=int(np.ceil(m/batch_size))

def fetch_batch(epoch,batch_index,batch_size):
    [...]#load data 
    return X_batch,y_batch
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_pochs):
        for batch_index in range(n_batches):
            X_batch,y_batch=fetch_batch(epoch,batch_index,batch_size)
            if batch_index%10==0:
                summary_str=mse_summary.eval(feed_dict={X:X_batch,y:y_batch})
                step=epoch*n_batches+batch_index
                file_writer.add_summary(summary_str,step)
                #可以在训练中每隔一段时间将数据保存下来
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
            best_theta=theta.eval()
            
            file_writer.close()#必须结束写入命令
#ls -l tf_log/run*查看文件
#tensorboard 进入tensorboard

#show_graph deepdream tutorial notebook
#TensorFlow debugger tool

#命名域
#在处理复杂模型时，计算图中可能有很多节点，为避免这种状况，需定义命名域.
#定义命名域后，节点将会在tensorboard中以二级菜单的方式隐藏
with tf.name_scope('loss') as scope:
    error=y_pred-y
    mse=tf.reduce_mean(tf.square(error),name='mse')
    
#模块化modularity
n_features=3
X=tf.placeholder(tf.float32,shape=(None,n_feature),name='X')
w1=tf.Variable(tf.ramdom_normal((n_feature,1)),name='weights1')
w2=tf.Variable(tf.random_normal((n_feature,1)),name='weights2')
b1=tf.Variable(0.0,name='bias1')
b2=tf.Variable(0.0,name='bias2')
z1=tf.add(tf.matmul(X,w1),b1,name='z1')
z2=tf.add(tf.matmul(X,w2),b2,name='z2')
relu1=tf.maximum(z1,0,name='relu1')
relu2=tf.maximum(z2,0,name='relu2')
output=tf.add(relu1,relu2,name='output')

#通过模块化减少代码重复率
#tensorflow会自动检查命名是否已出现，如未出现则命名之，已出现则加下标。

def relu(X):
    with tf.name_scope('relu'):#增加命名空间使可视化效果更简洁
        w_shape=(int(X.get_shape()[1]),1)
        w=tf.Variable(tf.random_normal(w_shape),name='weights')
        b=tf.Variable(0.0,name='bias')
        z=tf.add(matmul(X,w),b,name='z')
        return tf.maximum(z,0.,name='relu')
n_features=3
X=tf.placeholder(tf.float32,shape=(None,n_features),name='X')
relus=[relu(X) for i in range(5)]
output=tf.add_n(relus,name='output')#支持组内元素相加

##共享变量
#首先定义函数，再定义变量，将变量传给函数
def relu(X,threshold):
    with tf.name_scope('relu'):
        w=tf.Variable(tf.random_normal(w_shape),name='weights')
        b=tf.Variable(0.0,name='bias')
        z=tf.add(matmul(X,w),b,name='z')
        return tf.maximum(z,threshold,name='relu')
threshold=tf.Variable(0.0,name='threshold')
X=tf.placeholder(tf.float32,shape=(None,n_features),name='X')
relus=[relu(X,threshold) for i in range(5)]
output=tf.add_n(relus,name='output')

#但以上方法必须每次都有传递参数，为避免这种状况。一种方法是将模型中的变量定义变量字典，并
#传递给所有函数。另一种方法是为每个模块定义一个类，还有一种方法是将共享变量定义成函数的属性。
def relu(X):
    with tf.name_scope('relu'):
        if not hasattr(relu,;threshold):
            relu.threshold=tf.Variable(0.0,name='threshold')
            w_shape = int(X.get_shape()[1]), 1                          # not shown in the book
            w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
            b = tf.Variable(0.0, name="bias")                           # not shown
            z = tf.add(tf.matmul(X, w), b, name="z")   
            return tf.maximum(z,relu.threshold,name='max')
#另一种比较常用的方法是get_variable()函数创建共享变量，如果不存在则创建；如果存在则重新调用
#该功能由variable_scope属性控制。
with tf.variable_scope('relu'):
    threshold=tf.get_variable('threshold',shape=(),initilizer=tf.constant_initilizer(0.0))
#如果变量已被get_variable(）定义，该方法会报错；如果要复用，则需要在定义变量域时申明可复用
with tf.variable_scope('relu',reuse=True):
    threshold=tf.get_variable('threshold')
#但如果变量不存在，或者不是由get_varialbe()定义，该方法会报错。
#也可以将reuse属性定义在variable_scope内部
with tf.variable_scope('relu') as scope:
    scope.reuse_variables()
    threshold=tf.get_variable('threshold')
#一旦reuse被设为True，就不可设为False，如果在一个variable scope定义其他variable scope，
#其他variable scope会自动继承reuse=True,只有get_variable（）定义的参数才能这样用。





[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]
[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


In [13]:
#先定义函数，函数中声名可以重用已存在变量，然后先定义变量，再调用函数。
import tensorflow as tf
tf.reset_default_graph()

def relu(X):
    with tf.variable_scope('relu',reuse=True):
        threshold=tf.get_variable('threshold')#重用已存在变量
        w_shape = int(X.get_shape()[1]), 1 
        w=tf.Variable(tf.random_normal(w_shape),name='weights')
        b=tf.Variable(0.0,name='bias')
        z=tf.add(tf.matmul(X,w),b,name='z')
        return tf.maximum(z,threshold,name='max')
X=tf.placeholder(tf.float32,shape=(None,n_features),name='X')
with tf.variable_scope('relu'):#创建变量
    threshold=tf.get_variable('threshold',shape=(),initializer=tf.constant_initializer(0.0))
relus=[relu(X) for relu_index in range(5)]
output=tf.add_n(relus,name='output')
    #首先定义relu函数，然后创建relu/threshold变量，创建五个ReLU，relu()重用relu/threshold
    #变量，并创建其他ReLU节点。
    

    


In [28]:
#一些情况下，我们可能必须在relu之外定义threshold，而ReLU节点定义在relu()内部。
#以下代码在第一次调用时在relu()函数中创建threshold变量，在之后的调用时复用。
#relu()函数之后就可以调用get_variable()以调用或复用threshold变量。
#需控制在第一次调用时reuse=False
import tensorflow as tf
tf.reset_default_graph()
def relu(X):
    threshold=tf.get_variable('threshold',shape=(),initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)
    w=tf.Variable(tf.random_normal(w_shape),name='weights')
    b=tf.Variable(0.0,name='bias')
    z=tf.add(tf.matmul(X,w),b,name='z')
    return tf.maximum(z,threshold,name='max')
X=tf.placeholder(tf.float32,shape=(None,n_features),name='X')
relus=[]
for relu_index in range(5):
    with tf.variable_scope('relu',reuse=(relu_index>=1)) as scope:
        
        relus.append(relu(X))
output=tf.add_n(relus,name='output')
        

In [21]:
import tensorflow as tf
tf.reset_default_graph()
def relu(X):
    with tf.variable_scope('relu') as scope:
        scope.reuse_variables()
        threshold=tf.get_variable('threshold')#重用已存在变量
        w=tf.Variable(tf.random_normal((n_features,1)),name='weights')
        b=tf.Variable(0.0,name='bias')
        z=tf.add(tf.matmul(X,w),b,name='z')
        return tf.maximum(z,threshold,name='max')
n_features=3
X=tf.placeholder(tf.float32,shape=(None,n_features),name='X')

with tf.variable_scope('relu'):#创建变量
    
    threshold=tf.get_variable('threshold',shape=())
relus=[relu(X) for relu_index in range(5)]
output=tf.add_n(relus,name='output')


